<a href="https://colab.research.google.com/github/zackives/upenn-cis-2450/blob/main/21_Module_4_Part_III_Real_Time_Streams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Low-Level Streams


## Basic Setup

### Autograder setup

In [1]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 99999999 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

In [2]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Overwriting notebook-config.yaml


In [3]:
%set_env HW_ID=cis2450_fall24_HW9

env: HW_ID=cis2450_fall24_HW9


In [4]:
!pip3 install penngrader-client

In [5]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

PennGrader initialized with Student ID: 99999999

Make sure this correct or we will not be able to store your grade


## Installation of Storm, StreamParse and Their Support Libraries

In [6]:
%set_env STORM_VERSION=1.2.3

env: STORM_VERSION=1.2.3


In [7]:
!wget https://raw.githubusercontent.com/technomancy/leiningen/stable/bin/lein

--2024-11-26 13:59:06--  https://raw.githubusercontent.com/technomancy/leiningen/stable/bin/lein
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14217 (14K) [text/plain]
Saving to: ‘lein.1’

lein.1              100%[===================>]  13.88K  --.-KB/s    in 0.001s  

2024-11-26 13:59:06 (20.8 MB/s) - ‘lein.1’ saved [14217/14217]



In [8]:
!wget -nc https://archive.apache.org/dist/storm/apache-storm-$STORM_VERSION/apache-storm-$STORM_VERSION.tar.gz

File ‘apache-storm-1.2.3.tar.gz’ already there; not retrieving.



In [9]:
!tar zxvf apache-storm-$STORM_VERSION.tar.gz

apache-storm-1.2.3/lib/storm-core-1.2.3.jar
apache-storm-1.2.3/lib/kryo-3.0.3.jar
apache-storm-1.2.3/lib/reflectasm-1.10.1.jar
apache-storm-1.2.3/lib/asm-5.0.3.jar
apache-storm-1.2.3/lib/minlog-1.3.0.jar
apache-storm-1.2.3/lib/objenesis-2.1.jar
apache-storm-1.2.3/lib/clojure-1.7.0.jar
apache-storm-1.2.3/lib/ring-cors-0.1.5.jar
apache-storm-1.2.3/lib/disruptor-3.3.11.jar
apache-storm-1.2.3/lib/log4j-api-2.8.2.jar
apache-storm-1.2.3/lib/log4j-core-2.8.2.jar
apache-storm-1.2.3/lib/log4j-slf4j-impl-2.8.2.jar
apache-storm-1.2.3/lib/slf4j-api-1.7.21.jar
apache-storm-1.2.3/lib/log4j-over-slf4j-1.6.6.jar
apache-storm-1.2.3/lib/metrics-core-3.1.0.jar
apache-storm-1.2.3/lib/metrics-graphite-3.1.0.jar
apache-storm-1.2.3/lib/servlet-api-2.5.jar
apache-storm-1.2.3/lib/storm-rename-hack-1.2.3.jar
apache-storm-1.2.3/bin/storm-config.cmd
apache-storm-1.2.3/bin/storm.ps1
apache-storm-1.2.3/bin/storm-kafka-monitor
apache-storm-1.2.3/bin/storm.cmd
apache-storm-1.2.3/bin/storm.py
apache-storm-1.2.3/bin/st

In [10]:
!chmod a+x ./lein
!chmod a+x /content/apache-storm-$STORM_VERSION/bin/storm

In [11]:
%env PATH /opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/apache-storm-1.2.3/bin:/content


env: PATH=/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/apache-storm-1.2.3/bin:/content


In [12]:
!lein version

Leiningen 2.11.2 on Java 11.0.25 OpenJDK 64-Bit Server VM


In [13]:
!cp apache-storm-1.2.3/bin/storm apache-storm-1.2.3/bin/storm.bak
!cat apache-storm-1.2.3/bin/storm.bak | sed 's/bin\/python2.6/local\/bin\/python/' > apache-storm-1.2.3/bin/storm

In [14]:
!storm version

Running: java -client -Ddaemon.name= -Dstorm.options= -Dstorm.home=/content/apache-storm-1.2.3 -Dstorm.log.dir=/content/apache-storm-1.2.3/logs -Djava.library.path=/usr/local/lib:/opt/local/lib:/usr/lib -Dstorm.conf.file= -cp /content/apache-storm-1.2.3/*:/content/apache-storm-1.2.3/lib/*:/content/apache-storm-1.2.3/extlib/*:/content/apache-storm-1.2.3/extlib-daemon/*:/content/apache-storm-1.2.3/conf org.apache.storm.utils.VersionInfo
Storm 1.2.3
URL https://git-wip-us.apache.org/repos/asf/storm.git -r 4e162a47c8219546ab9639401363a8f1b5e51119
Branch (no branch)
Compiled by tgoetz on 2019-05-04T22:44Z
From source with checksum a43d33df1c96ae6f325ef011af860a0


In [15]:
!pip install streamparse

## Initial Configuration and Execution

Here you can see Streamparse in action, using a sample word count topology.

In [16]:
import os

os.environ['LEIN_ROOT']='1'

In [17]:
!sparse quickstart wordcount

/usr/local/lib/python3.10/dist-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/usr/local/lib/python3.10/dist-packages/paramiko/transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
error: folder "wordcount" already exists


We need to make a quick fix to tell StreamParse to use Storm 1.2.3 instead of 1.1.0...

In [18]:
!cat wordcount/project.clj | sed s/1.1.0/$STORM_VERSION/ > wordcount/project2.clj

In [19]:
!cat wordcount/project2.clj

(defproject wordcount "0.0.1-SNAPSHOT"
  :resource-paths ["_resources"]
  :target-path "_build"
  :min-lein-version "2.0.0"
  :jvm-opts ["-client"]
  :dependencies  [[org.apache.storm/storm-core "1.2.3"]
                  [org.apache.storm/flux-core "1.2.3"]]
  :jar-exclusions     [#"log4j\.properties" #"org\.apache\.storm\.(?!flux)" #"trident" #"META-INF" #"meta-inf" #"\.yaml"]
  :uberjar-exclusions [#"log4j\.properties" #"org\.apache\.storm\.(?!flux)" #"trident" #"META-INF" #"meta-inf" #"\.yaml"]
  )
(require 'cemerick.pomegranate.aether)
(cemerick.pomegranate.aether/register-wagon-factory!
 "http" #(org.apache.maven.wagon.providers.http.HttpWagon.))


In [20]:
%%writefile project.cl_sec

(require 'cemerick.pomegranate.aether)
(cemerick.pomegranate.aether/register-wagon-factory!
 "http" #(org.apache.maven.wagon.providers.http.HttpWagon.))

Overwriting project.cl_sec


In [21]:
!cat wordcount/project2.clj project.cl_sec > wordcount/project.clj

## Running StreamParse for Real

Run this code and see the output...  At some point you'll want to interrupt it.  Press [Ctrl]-[M][I] when you are done.

In [22]:
!cat /content/wordcount/project.clj

(defproject wordcount "0.0.1-SNAPSHOT"
  :resource-paths ["_resources"]
  :target-path "_build"
  :min-lein-version "2.0.0"
  :jvm-opts ["-client"]
  :dependencies  [[org.apache.storm/storm-core "1.2.3"]
                  [org.apache.storm/flux-core "1.2.3"]]
  :jar-exclusions     [#"log4j\.properties" #"org\.apache\.storm\.(?!flux)" #"trident" #"META-INF" #"meta-inf" #"\.yaml"]
  :uberjar-exclusions [#"log4j\.properties" #"org\.apache\.storm\.(?!flux)" #"trident" #"META-INF" #"meta-inf" #"\.yaml"]
  )
(require 'cemerick.pomegranate.aether)
(cemerick.pomegranate.aether/register-wagon-factory!
 "http" #(org.apache.maven.wagon.providers.http.HttpWagon.))

(require 'cemerick.pomegranate.aether)
(cemerick.pomegranate.aether/register-wagon-factory!
 "http" #(org.apache.maven.wagon.providers.http.HttpWagon.))


In [23]:
!cd /content/wordcount ; sparse run

/usr/local/lib/python3.10/dist-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/usr/local/lib/python3.10/dist-packages/paramiko/transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
Cleaning from prior builds...
Creating topology Uber-JAR...
Uber-JAR created: /content/wordcount/_build/wordcount-0.0.1-SNAPSHOT-standalone.jar
Removing _resources temporary directory...done
[localhost] local: storm jar /content/wordcount/_build/wordcount-0.0.1-SNAPSHOT-standalone.jar org.apache.storm.flux.Flux --local --no-splash --sleep 9223372036854775807 /tmp/tmpoj_oljw8.yaml
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/con

# Streaming, Joins, and Learning

Let's take a look at how we might use StreamParse spouts and bolts to implement a stream-based incremental machine learning application, for our airline flight data from the High-Level Streaming application.

Here we'll use a combination of Pandas (and PandaSQL) for our incremental joins and grouping, and Creme for incremental machine learning!

For the rest of this code, rather than running directly in a StreamParse topology we'll run operator-by-operator to demonstrate the functionality.

In [24]:
!pip install duckdb
!pip install river

Let's get our data...

In [25]:
!wget -nc https://storage.googleapis.com/penn-cis5450/airlines.csv
!wget -nc https://storage.googleapis.com/penn-cis5450/airports.csv
!wget -nc https://storage.googleapis.com/penn-cis5450/ontime.csv
#!wget -nc https://storage.googleapis.com/penn-cis5450/2015-ontime.csv

File ‘airlines.csv’ already there; not retrieving.

File ‘airports.csv’ already there; not retrieving.

File ‘ontime.csv’ already there; not retrieving.



## Our Spout

Let's define a spout.  To actually run this in StreamParse we would need to put it in a separate source file, which is inconvenient on Colab.

So instead we'll simulate StreamParse and watch what happens.

This spout reads one row at a time from the `ontime` dataframe.

In [26]:
!mkdir spouts

mkdir: cannot create directory ‘spouts’: File exists


In [27]:
%%writefile spouts/flight_spout.py
from streamparse.spout import Spout
import pandas as pd
import duckdb

class FlightSpout(Spout):
  outputs=['YEAR','MONTH','DAY_OF_MONTH','AIRLINE_ID','CARRIER','FL_NUM','ORIGIN','DEST','ARR_DELAY','CANCELLED']
  # Position of next tuple in dataframe
  inx = 0
  # Overall stream tuple id
  tid = 0

  env = False

  # Open the file as a dataframe
  def initialize(self, stormconf, context):
    self.ontime_df = pd.read_csv('/content/ontime.csv')
    self.ontime_df.dropna(inplace=True,subset=['ARR_DELAY_NEW'])
    print('Loaded {} entries'.format(len(self.ontime_df)))
    self.env = (context is not None)
    self.inx = 0

  # Used for showing an example
  def set_stream(self, stream):
    self.stream = stream

  # Read one row, increment the pointer, return the
  # row as a list
  def next_tuple(self):
    tup = self.ontime_df.iloc[self.inx]
    self.inx = self.inx + 1

    # We'll wrap around if we exceed the size of the table
    if self.inx >= len(self.ontime_df):
      self.inx = 0

    # We have to turn the series into a list of columns.
    # There is also an extra, blank column in the file
    self.emit(tup.tolist()[0:-1], tup_id=self.tid)
    self.tid = self.tid + 1

  # If a tuple was processed properly, do nothing
  def ack(self, tup_id):
    pass

  # If a tuple was processed incorrectly, we'll still
  # do nothing
  def fail(self, tup_id):
    pass

  # This lets us run outside Streamparse
  def emit(self, tuple, tup_id):
    if not self.env:
      self.stream.append(tuple)
    else:
      super().emit(tuple, tup_id)


Overwriting spouts/flight_spout.py


## A Bolt that Joins

As we get tuples, we'll want to join the flight with latitude and longitude information for origin and destination.  We can do this by preloading the `airports` as a dataframe, and incrementally joining tuples as they arrive.

In [28]:
!mkdir bolts

mkdir: cannot create directory ‘bolts’: File exists


In [29]:
%%writefile bolts/join_bolt.py
from streamparse.bolt import Bolt
from pystorm.component import Tuple
import pandas as pd
import duckdb

class JoinBolt(Bolt):
  outputs=['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST','FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','ARR_DELAY']
  env = False
  inx = 0

  def initialize(self, stormconf, context):
    self.airports_df = pd.read_csv('/content/airports.csv')
    self.env = (context is not None)
    self.inx = 0

  def process(self, tup):
    flights_df = pd.DataFrame([tup],columns=['YEAR','MONTH','DAY_OF_MONTH','AIRLINE_ID','CARRIER','FL_NUM','ORIGIN','DEST','ARR_DELAY','CANCELLED'])
    airports_df = self.airports_df
    incremental_results = \
      duckdb.query('select YEAR,MONTH,DAY_OF_MONTH,CARRIER,FL_NUM, ORIGIN, DEST, org.LATITUDE AS FROM_LAT, '\
            'org.LONGITUDE AS FROM_LONG, '\
            'dst.LATITUDE AS TO_LAT, dst.LONGITUDE AS TO_LONG, ARR_DELAY '\
            'from flights_df f join airports_df org on f.origin=org.IATA_CODE '\
            'join airports_df dst on f.dest=dst.IATA_CODE').to_df()
    self.emit(incremental_results.iloc[0].tolist(), self.inx)
    self.inx += 1

  # Used for showing an example
  def set_stream(self, stream):
    self.stream = stream


  # This lets us run outside Streamparse
  def emit(self, tuple, tup_id):
    if not self.env:
      self.stream.append(tuple)
    else:
      super().emit(tuple, tup_id)


Overwriting bolts/join_bolt.py


## Windowed Grouping

Let's group by day.  We are going to assume that the stream is well-behaved in that days are monotonically increasing in the stream.

Thus, we buffer up tuples as long as they have the same day.  Once the day switches, we group them by the day and compute our statistics, then emit that.

In [30]:
%%writefile bolts/group_bolt.py
from streamparse.bolt import Bolt
from pystorm.component import Tuple
import pandas as pd
import duckdb

class GroupBolt(Bolt):
  outputs = ['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST',\
                             'FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','NBR_FLIGHTS','AVG_DELAY']
  env = False
  inx = 0

  def initialize(self, stormconf, context):
    self.groups = pd.DataFrame([],
                               columns=['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST','FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','NBR_FLIGHTS','ARR_DELAY'])
    self.last_year = None
    self.last_month = None
    self.last_day_of_month = None
    self.env = (context is not None)
    self.inx = 0

  # Used for showing an example
  def set_stream(self, stream):
    self.stream = stream


  def process(self, tup):
    tup2 = tup[0:]
    if len(tup2) < 13:
      tup2.append(tup[-1])
      tup2[len(tup2)-2] = 1
    # print ("Input: {}".format(len(tup2)))
    flights_df = pd.DataFrame([tup2],
                              columns=['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST','FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','NBR_FLIGHTS','ARR_DELAY'])

    # print ('Flights {}'.format(len(flights_df.columns)))
    if flights_df.iloc[0,0] != self.last_year or flights_df.iloc[0,1] != self.last_month \
      or flights_df.iloc[0,2] != self.last_day_of_month:
      print('** DONE WITH {}-{}-{} with {} entries'.format(self.last_year,self.last_month,self.last_day_of_month,len(self.groups)))
      if len(self.groups) > 0:
        groups_df = self.groups
        grouped_results = duckdb.query('''
                        select YEAR,MONTH,DAY_OF_MONTH,CARRIER,FL_NUM,ORIGIN,DEST,
                        FROM_LAT,FROM_LONG,TO_LAT,TO_LONG, COUNT(ARR_DELAY) as NBR_FLIGHTS, avg(ARR_DELAY) as ARR_DELAY
                        FROM groups_df GROUP BY YEAR,MONTH,DAY_OF_MONTH,CARRIER,FL_NUM,ORIGIN,DEST,FROM_LAT,FROM_LONG,TO_LAT,TO_LONG
                        ORDER BY YEAR,MONTH,DAY_OF_MONTH,CARRIER,FL_NUM,ORIGIN,DEST''').to_df()

        for result in grouped_results.itertuples(index=False):
          self.emit(list(result), self.inx)
          self.inx += 1

      self.groups = flights_df
      self.last_year = flights_df.iloc[0,0]
      self.last_month = flights_df.iloc[0,1]
      self.last_day_of_month = flights_df.iloc[0,2]
    else:
      # print ('Groups {}'.format(len(self.groups.columns)))
      self.groups = pd.concat([self.groups, flights_df])

  # This lets us run outside Streamparse
  def emit(self, tuple, tup_id):
    if not self.env:
      self.stream.append(tuple)
    else:
      super().emit(tuple, tup_id)


Overwriting bolts/group_bolt.py


## Incremental Learning via Creme (now part of River)

We want to incrementally train a linear regression algorithm to predict our delays.

The Creme / River package (https://github.com/online-ml/river) allows us to do incrementally process data, using stochastic gradient descent-style methods to incrementally perform linear regression.  It also has incremental methods for one-hot encoding, scaling, etc.

In [31]:
%%writefile bolts/learn_bolt.py
from streamparse.bolt import Bolt
from pystorm.component import Tuple
import pandas as pd
import river.preprocessing
import river.linear_model

class LearnBolt:#(Bolt):
  outputs = ['predicted','actual','error']
  env = False
  inx = 0

  def initialize(self, stormconf, context):
    self.scaler = river.preprocessing.StandardScaler()
    self.lin_reg = river.linear_model.LinearRegression()
    self.carrier_one_hot = river.preprocessing.OneHotEncoder('CARRIER')
    self.origin_one_hot = river.preprocessing.OneHotEncoder('ORIGIN')
    self.dest_one_hot = river.preprocessing.OneHotEncoder('DEST')
    self.env = (context is not None)

  # Used for showing an example
  def set_stream(self, stream):
    self.stream = stream


  def process(self, tup):
    x = pd.Series(tup, index=['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST',\
                             'FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','NBR_FLIGHTS','AVG_DELAY'])

    y = x['AVG_DELAY']
    x = x.drop('AVG_DELAY')
    oh1 = self.carrier_one_hot
    oh1.learn_one(x.to_dict())
    x_onehot = oh1.transform_one(x.to_dict())
    x = x.drop('CARRIER')
    oh2 = self.origin_one_hot
    oh2.learn_one(x)
    x_onehot_1 = oh2.transform_one(x)
    x = x.drop('ORIGIN')
    oh3 = self.dest_one_hot
    oh3.learn_one(x)
    x_onehot_2 = oh3.transform_one(x)
    x = x.drop('DEST')

    # x = x.drop(['CARRIER','AVG_DELAY','ORIGIN','DEST'])
    x = pd.concat([x, pd.Series(x_onehot_1), pd.Series(x_onehot_2)])

    oh4 = self.scaler
    oh4.learn_one(x)
    x = oh4.transform_one(x)
    yhat = self.lin_reg.predict_one(x)
    print('Predicted {}, actual {}, error {}'.format(yhat, y, abs(yhat - y)))
    self.emit([yhat,y,abs(yhat-y)], self.inx)

    self.lin_reg.learn_one(x, y)
    self.inx += 1


  # This lets us run outside Streamparse
  def emit(self, tuple, tup_id):
    if not self.env:
      self.stream.append(tuple)
    else:
      super().emit(tuple, tup_id)


Overwriting bolts/learn_bolt.py


In [32]:
from spouts.flight_spout import FlightSpout
from bolts.join_bolt import JoinBolt
from bolts.group_bolt import GroupBolt
from bolts.learn_bolt import LearnBolt
import pandas as pd

stream = []
fs = FlightSpout()

fs.initialize(None, None)
fs.set_stream(stream)
for i in range(2000):
  fs.next_tuple()

pd.DataFrame(stream,columns=['YEAR','MONTH','DAY_OF_MONTH','AIRLINE_ID','CARRIER','FL_NUM','ORIGIN','DEST','ARR_DELAY','CANCELLED'])

Loaded 551498 entries


,YEAR,MONTH,DAY_OF_MONTH,AIRLINE_ID,CARRIER,FL_NUM,ORIGIN,DEST,ARR_DELAY,CANCELLED
0,2018,1,2,19393,WN,1325,SJU,MCO,0.0,0.0
1,2018,1,2,19393,WN,5159,SJU,MCO,0.0,0.0
2,2018,1,2,19393,WN,5890,SJU,MCO,9.0,0.0
3,2018,1,2,19393,WN,6618,SJU,MCO,0.0,0.0
4,2018,1,2,19393,WN,1701,SJU,MDW,8.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1995,2018,1,3,19393,WN,762,LAS,OAK,55.0,0.0
1996,2018,1,3,19393,WN,1029,LAS,OAK,0.0,0.0
1997,2018,1,3,19393,WN,1358,LAS,OAK,18.0,0.0
1998,2018,1,3,19393,WN,1396,LAS,OAK,0.0,0.0


In [33]:
stream2 = []
jb = JoinBolt()

jb.initialize(None, None)
jb.set_stream(stream2)

for e in stream:
  jb.process(e)

pd.DataFrame(stream2,columns=['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST','FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','ARR_DELAY'])

,YEAR,MONTH,DAY_OF_MONTH,CARRIER,FL_NUM,ORIGIN,DEST,FROM_LAT,FROM_LONG,TO_LAT,TO_LONG,ARR_DELAY
0,2018,1,2,WN,1325,SJU,MCO,18.43942,-66.00183,28.42889,-81.31603,0.0
1,2018,1,2,WN,5159,SJU,MCO,18.43942,-66.00183,28.42889,-81.31603,0.0
2,2018,1,2,WN,5890,SJU,MCO,18.43942,-66.00183,28.42889,-81.31603,9.0
3,2018,1,2,WN,6618,SJU,MCO,18.43942,-66.00183,28.42889,-81.31603,0.0
4,2018,1,2,WN,1701,SJU,MDW,18.43942,-66.00183,41.78598,-87.75242,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2018,1,3,WN,762,LAS,OAK,36.08036,-115.15233,37.72129,-122.22072,55.0
1996,2018,1,3,WN,1029,LAS,OAK,36.08036,-115.15233,37.72129,-122.22072,0.0
1997,2018,1,3,WN,1358,LAS,OAK,36.08036,-115.15233,37.72129,-122.22072,18.0
1998,2018,1,3,WN,1396,LAS,OAK,36.08036,-115.15233,37.72129,-122.22072,0.0


In [34]:
stream3 = []
gb = GroupBolt()

gb.initialize(None, None)
gb.set_stream(stream3)

for e in stream2:
  gb.process(e)

pd.DataFrame(stream3,columns=['YEAR','MONTH','DAY_OF_MONTH','CARRIER','FL_NUM','ORIGIN','DEST',\
                             'FROM_LAT','FROM_LONG','TO_LAT','TO_LONG','NBR_FLIGHTS','AVG_DELAY'])

** DONE WITH None-None-None with 0 entries
** DONE WITH 2018-1-2 with 383 entries


,YEAR,MONTH,DAY_OF_MONTH,CARRIER,FL_NUM,ORIGIN,DEST,FROM_LAT,FROM_LONG,TO_LAT,TO_LONG,NBR_FLIGHTS,AVG_DELAY
0,2018,1,2,WN,23,SMF,PDX,38.69542,-121.59077,45.58872,-122.59750,1,0.0
1,2018,1,2,WN,24,TPA,HOU,27.97547,-82.53325,29.64542,-95.27889,1,0.0
2,2018,1,2,WN,28,TPA,MSY,27.97547,-82.53325,29.99339,-90.25803,1,5.0
3,2018,1,2,WN,29,SNA,DEN,33.67566,-117.86822,39.85841,-104.66700,1,19.0
4,2018,1,2,WN,50,STL,MCO,38.74769,-90.35999,28.42889,-81.31603,1,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,2018,1,2,WN,6963,SNA,OAK,33.67566,-117.86822,37.72129,-122.22072,1,25.0
379,2018,1,2,WN,6973,STL,DAL,38.74769,-90.35999,32.84711,-96.85177,1,8.0
380,2018,1,2,WN,6986,STL,BNA,38.74769,-90.35999,36.12448,-86.67818,1,10.0
381,2018,1,2,WN,6997,STL,DAL,38.74769,-90.35999,32.84711,-96.85177,1,0.0


In [35]:
stream4 = []
lb = LearnBolt()

lb.initialize(None, None)
lb.set_stream(stream4)

for e in stream3:
  lb.process(e)

pd.DataFrame(stream4,columns=['predicted','actual','error'])

Predicted 0.0, actual 0.0, error 0.0
Predicted 0.0, actual 0.0, error 0.0
Predicted 0.0, actual 5.0, error 5.0
Predicted 0.12804444299265683, actual 19.0, error 18.871955557007343
Predicted 1.4804655320542017, actual 6.0, error 4.519534467945798
Predicted 1.3289121609584666, actual 0.0, error 1.3289121609584666
Predicted 2.5130539013922144, actual 0.0, error 2.5130539013922144
Predicted 1.7947495397727091, actual 0.0, error 1.7947495397727091
Predicted 1.3150340887675878, actual 0.0, error 1.3150340887675878
Predicted 1.3878920768967375, actual 0.0, error 1.3878920768967375
Predicted 1.0450065090872644, actual 6.0, error 4.954993490912735
Predicted 2.104364305768828, actual 0.0, error 2.104364305768828
Predicted 1.7139290058209553, actual 6.0, error 4.2860709941790445
Predicted 2.2655952050576196, actual 0.0, error 2.2655952050576196
Predicted 1.3847346078865956, actual 0.0, error 1.3847346078865956
Predicted 0.968417613002726, actual 1.0, error 0.031582386997274
Predicted 0.9348007878

,predicted,actual,error
0,0.000000,0.0,0.000000
1,0.000000,0.0,0.000000
2,0.000000,5.0,5.000000
3,0.128044,19.0,18.871956
4,1.480466,6.0,4.519534
...,...,...,...
378,7.447098,25.0,17.552902
379,9.305029,8.0,1.305029
380,10.915461,10.0,0.915461
381,9.169343,0.0,9.169343


So far we've looked at the bolts separately, just to see what was on the streams.  Let's see them all together in a topology!

In [36]:
!mkdir topologies

mkdir: cannot create directory ‘topologies’: File exists


In [37]:
%%writefile topologies/flight_topology.py
from streamparse import Grouping, Topology


class ModelFlights(Topology):
  flight_spout = FlightSpout.spec()
  join_bolt = JoinBolt.spec(inputs={flight_spout: Grouping.fields("YEAR","MONTH","DAY_OF_MONTH")}, par=2)
  group_bolt = GroupBolt.spec(inputs={join_bolt: Grouping.fields("YEAR","MONTH","DAY_OF_MONTH")}, par=2)
  learn_bolt = LearnBolt.spec(inputs={join_bolt: Grouping.fields("CARRIER","FL_NUM","ORIGIN","DEST")}, par=2)

Overwriting topologies/flight_topology.py


In [ ]:
%%writefile config.json
{
  "serializer": "json",
  "topology_specs": "topologies/",
  "virtualenv_specs": "virtualenvs/",
  "envs": {
      "prod": {
          "user": "",
          "ssh_password": "",
          "nimbus": "",
          "workers": [],
          "log": {
              "path": "",
              "max_bytes": 1000000,
              "backup_count": 10,
              "level": "info"
          },
          "virtualenv_root": ""
      }
  }
}

## Exercise

Write a *filter bolt* that takes a set of input tuples with schema:

```
['YEAR','MONTH','DAY_OF_MONTH','AIRLINE_ID','CARRIER','FL_NUM','ORIGIN','DEST','ARR_DELAY','CANCELLED']
```

and only emits tuples that arrive in `PHL`.

In [38]:
%%writefile bolts/filter_bolt.py
from streamparse.bolt import Bolt
from pystorm.component import Tuple
import pandas as pd
import duckdb

class FilterBolt(Bolt):
  outputs=['YEAR','MONTH','DAY_OF_MONTH','AIRLINE_ID','CARRIER','FL_NUM','ORIGIN','DEST','ARR_DELAY','CANCELLED']
  env = False
  inx = 0

  def initialize(self, stormconf, context):
    self.env = (context is not None)
    self.inx = 0

  def process(self, tup):
    # TODO: filter and then emit!

    self.inx += 1

  # Used for showing an example
  def set_stream(self, stream):
    self.stream = stream


  # This lets us run outside Streamparse
  def emit(self, tuple, tup_id):
    if not self.env:
      self.stream.append(tuple)
    else:
      super().emit(tuple, tup_id)


Overwriting bolts/filter_bolt.py


Here's some test code for your bolt.

In [ ]:
from bolts.filter_bolt import FilterBolt

stream5 = []
fb = FilterBolt()

fb.initialize(None, None)
fb.set_stream(stream5)

for e in stream:
  fb.process(e)

pd.DataFrame(stream5,columns=fb.outputs)

In [ ]:
from dill.source import getsource

grader.grade('filter_bolt', getsource(FilterBolt.process))